# Wstęp

Metody uczenia maszynowego możemy podzielić na dwie główne kategorie (pomijając uczenie ze wzmocnieniem): nadzorowane i nienadzorowane. Uczenie **nadzorowane** (ang. *supervised*) to jest uczenie z dostępnymi etykietami dla danych wejściowych. Na parach danych uczących $dataset= \{(x_0,y_0), (x_1,y_1), \ldots, (x_n,y_n)\}$ model ma za zadanie nauczyć się funkcji $f: X \rightarrow Y$. Z kolei modele uczone w sposób **nienadzorowany** (ang. *unsupervised*) wykorzystują podczas trenowania dane nieetykietowane tzn. nie znamy $y$ z pary $(x, y)$.

Dość częstą sytuacją, z jaką mamy do czynienia, jest posiadanie małego podziobioru danych etykietowanych i dużego nieetykietowanych. Często annotacja danych wymaga ingerencji człowieka - ktoś musi określić co jest na obrazku, ktoś musi powiedzieć czy dane słowo jest rzeczownkiem czy czasownikiem itd.

Jeżeli mamy dane etykietowane do zadania uczenia nadzorowanego (np. klasyfikacja obrazka), ale także dużą ilość danych nieetykietowanych, to możemy wtedy zastosować techniki **uczenia częściowo nadzorowanego** (ang. *semi-supervised learning*). Te techniki najczęściej uczą się funkcji $f: X \rightarrow Y$, ale jednocześnie są w stanie wykorzystać informacje z danych nieetykietowanych do poprawienia działania modelu.

## Cel ćwiczenia

Celem ćwiczenia jest nauczenie modelu z wykorzystaniem danych etykietowanych i nieetykietowanych ze zbioru STL10 z użyciem metody [Bootstrap your own latent](https://arxiv.org/abs/2006.07733).

Metoda ta jest relatywnie "lekka" obliczeniowo, a także dość prosta do zrozumienia i zaimplementowania, dlatego też na niej się skupimy na tych laboratoriach.

# Zbiór STL10

Zbiór STL10 to zbiór stworzony i udostępniony przez Stanford [[strona]](https://ai.stanford.edu/~acoates/stl10/) [[papier]](https://cs.stanford.edu/~acoates/papers/coatesleeng_aistats_2011.pdf) a inspirowany przez CIFAR-10. Obrazy zostały pozyskane z [ImageNet](https://image-net.org/). Szczegóły można doczytać na ich stronie. To co jest ważne to to, że autorzy zbioru dostarczają predefiniowany plan eksperymentalny, żeby móc porównywać łatwo wyniki eksperymentów. Nie będziemy go tutaj stosować z uwagi na jego czasochłonność (10 foldów), ale warto pamiętać o tym, że często są z góry ustalone sposoby walidacji zaprojetowanych przez nas algorytmów na określonych zbiorach referencyjnych.

Korzystając z `torchvision.datasets` ***załaduj*** 3 podziały zbioru danych STL10: `train`, `test`, `unlabeled` oraz utwórz z nich instancje klasy `DataLoader`. Korzystając z Google Colab rozważ użycie Google Drive do przechowyania zbioru w calu zaoszczędzenia czasu na wielokrotne pobieranie.

In [1]:
import torch

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import STL10
# Definiuję ścieżkę do mojego katalogu na Google Drive
data_dir = "/content/drive/MyDrive/stl10_data"

transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# Załaduj zbiory train, test i unlabeled
train_dataset = datasets.STL10(root=data_dir, split='train', transform=transform, download=False)
test_dataset = datasets.STL10(root=data_dir, split='test', transform=transform, download=False)
# unlabeled_dataset = datasets.STL10(root=data_dir, split='unlabeled', transform=transform, download=False)

# Utwórz DataLoader dla każdego podziału
train_dl = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=128, shuffle=False)
# unlabeled_dl = DataLoader(unlabeled_dataset, batch_size=128, shuffle=False)

# Uczenie nadzorowane

Żeby porównać czy metoda BYOL przynosi nam jakieś korzyści musimy wyznaczyć wartość bazową metryk(i) jakości, których będziemu używać (np. dokładność).

***Zaimplementuj*** wybraną metodę uczenia nadzorowanego na danych `train` z STL10. Możesz wykorzystać predefiniowane architektury w `torchvision.models` oraz kody źródłowe z poprzednich list.

In [2]:
from typing import Tuple
from torchvision import models
from torch import nn
from torch import optim

from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
resnet18 = models.resnet18(pretrained=False)
# vgg19.classifier[6] = nn.Linear(4096, 10, bias=True)  # STL10 ma 10 klas
# model = vgg19.to(device)

c:\Users\krzys\anaconda3\envs\GSN\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\krzys\anaconda3\envs\GSN\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
def count_correct(
    y_pred: torch.Tensor, y_true: torch.Tensor
) -> torch.Tensor:
    preds = torch.argmax(y_pred, dim=1)
    return (preds == y_true).float().sum()

def validate(
    model: nn.Module, 
    loss_fn: torch.nn.CrossEntropyLoss, 
    dataloader: DataLoader
) -> Tuple[torch.Tensor, torch.Tensor]:
    loss = 0
    correct = 0
    all = 0
    for X_batch, y_batch in dataloader:
        y_pred = model(X_batch.to(device))
        all += len(y_pred)
        loss += loss_fn(y_pred, y_batch.to(device)).sum()
        correct += count_correct(y_pred, y_batch.to(device))
    return loss / all, correct / all

def fit(
    model: nn.Module, optimiser: optim.Optimizer, 
    loss_fn: torch.nn.CrossEntropyLoss, train_dl: DataLoader, 
    val_dl: DataLoader, epochs: int, 
    print_metrics: str = True
):
  for epoch in range(epochs):
      for X_batch, y_batch in tqdm(train_dl):
          y_pred = model(X_batch.to(device))
          loss = loss_fn(y_pred, y_batch.to(device))

          loss.backward()
          optimiser.step()
          optimiser.zero_grad()

      if print_metrics: 
          model.eval()
          with torch.no_grad():
              train_loss, train_acc = validate(
                  model=model, loss_fn=loss_fn, dataloader=train_dl
              ) 
              val_loss, val_acc = validate(
                  model=model, loss_fn=loss_fn, dataloader=val_dl
              )
              print(
                  f"Epoch {epoch}: "
                  f"train loss = {train_loss:.3f} (acc: {train_acc:.3f}), "
                  f"validation loss = {val_loss:.3f} (acc: {val_acc:.3f})"
              )

In [4]:
class ResNetSTL10(nn.Module):
    def __init__(self, num_classes):
        super(ResNetSTL10, self).__init__()
        resnet = models.resnet18(pretrained=False)
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Linear(num_ftrs, num_classes)
        self.resnet = resnet

    def forward(self, x):
        return self.resnet(x)

# Liczba klas w zbiorze danych STL10
num_classes = 10

In [5]:
model_finetuned = ResNetSTL10(10)
model_finetuned.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_finetuned = torch.optim.Adam(model_finetuned.parameters(), lr=0.001)
fit(model_finetuned, optimizer_finetuned, criterion, train_dl, test_dl, epochs=10)

100%|██████████| 40/40 [00:05<00:00,  6.81it/s]


Epoch 0: train loss = 0.016 (acc: 0.380), validation loss = 0.017 (acc: 0.345)


100%|██████████| 40/40 [00:03<00:00, 10.47it/s]


Epoch 1: train loss = 0.013 (acc: 0.365), validation loss = 0.013 (acc: 0.354)


100%|██████████| 40/40 [00:03<00:00, 10.98it/s]


...

Epoch 7: train loss = 0.009 (acc: 0.573), validation loss = 0.011 (acc: 0.490)


100%|██████████| 40/40 [00:03<00:00, 11.02it/s]


Epoch 8: train loss = 0.011 (acc: 0.522), validation loss = 0.013 (acc: 0.435)


100%|██████████| 40/40 [00:03<00:00, 11.01it/s]


Epoch 9: train loss = 0.009 (acc: 0.606), validation loss = 0.012 (acc: 0.504)


# Bootstrap your own latent

Metoda [Bootstrap your own latent](https://arxiv.org/abs/2006.07733) jest opisana w rodziale 3.1 papieru a także w dodatku A. Składa się z dwóch etapów:


1.   uczenia samonadzorowanego (ang. *self-supervised*)
2.   douczania nadzorowanego (ang. *fine-tuning*)

## Uczenie samonadzorowane

Architektura do nauczania samonadzorowanego składa się z dwóch sieci: (1) *online* i (2) *target*. W uproszczeniu cała architektura działa tak:


1.   Dla obrazka $x$ wygeneruj dwie różne augmentacje $v$ i $v'$ za pomocą funkcji $t$ i $t'$.
2.   Widok $v$ przekazujemy do sieci *online*, a $v'$ do *target*.
3.   Następnie widoki przekształacamy za pomocą sieci do uczenia reprezentacji (np. resnet18 lub resnet50) do reprezentacji $y_\theta$ i $y'_\xi$.
4.   Potem dokonujemy projekcji tych reprezentacji w celu zmniejszenia wymiarowości (np. za pomocą sieci MLP).
5.   Na sieci online dokonujmey dodatkowo predykcji pseudo-etykiety (ang. *pseudolabel*)
6.   Wyliczamy fukncję kosztu: MSE z wyjścia predyktora sieci *online* oraz wyjścia projekcji sieci *target* "przepuszczonej" przez predyktor sieci *online* **bez propagacji wstecznej** (*vide Algorithm 1* z papieru).
7.   Dokonujemy wstecznej propagacji **tylko** po sieci *online*.
8.   Aktualizujemy wagi sieci *target* sumując w ważony sposób wagi obu sieci $\xi = \tau\xi + (1 - \tau)\theta$ ($\tau$ jest hiperprametrem) - jest to ruchoma średnia wykładnicza (ang. *moving exponential average*).

Po zakończeniu procesu uczenia samonadzorowanego zostawiamy do douczania sieć kodera *online* $f_\theta$. Cała sieć *target* oraz warstwy do projekcji i predykcji w sieci *online* są "do wyrzucenia".

### Augmentacja

Dodatek B publikacji opisuje augmentacje zastosowane w metodzie BYOL. Zwróć uwagę na tabelę 6 w publikacji. `torchvision.transforms.RandomApply` może być pomocne.

***Zaimeplementuj*** augmentację $\tau$ i $\tau'$.


In [7]:
augmentation_t = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.2, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)], p=0.8),
    transforms.RandomApply([transforms.RandomGrayscale(p=1.0)], p=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=23, sigma=(0.1, 2.0))], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

augmentation_t_prime = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.2, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)], p=0.8),
    transforms.RandomApply([transforms.RandomGrayscale(p=1.0)], p=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=23, sigma=(0.1, 2.0))], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
stl10_ds_unlabeled_transformed = STL10(
    root=data_dir,
    split='unlabeled',
    transform=augmentation_t,
    download=True
)


stl10_ds_unlabeled_transformed_prime = STL10(
    root=data_dir,
    split='unlabeled',
    transform=augmentation_t_prime,
    download=True
)

transform_t_loader = DataLoader(
    stl10_ds_unlabeled_transformed,
    batch_size=128,
    shuffle=True,
    num_workers=2,
)

transform_t_prime_loader = DataLoader(
    stl10_ds_unlabeled_transformed_prime,
    batch_size=128,
    shuffle=True,
    num_workers=2,
)

Files already downloaded and verified
Files already downloaded and verified


### Implementacja uczenia samonadzorowanego

***Zaprogramuj*** proces uczenia samonadzorowanego na danych nieetykietowanych ze zbioru STL10.

Wskazówki do realizacji polecenia:

1. Proces uczenia może trwać bardzo długo dlatego zaleca się zastsowanie wczesnego zatrzymania lub uczenia przez tylko jedną epokę. Mimo wszystko powinno się dać osiągnąć poprawę w uczeniu nadzorowanym wykorzystując tylko zasoby z Google Colab.
2. Dobrze jest pominąć walidację na zbiorze treningowym i robić ją tylko na zbiorze walidacyjnym - zbiór treningowy jest ogromny i w związku z tym narzut czasowy na walidację też będzie duży.
3. Walidację modelu można przeprowadzić na zbiorze `train` lub całkowicie ją pominąć, jeżeli uczymy na stałej ilości epok.
4. Rozważ zastosowanie tylko jednej augmentacji - augmentacja $\tau'$ jest bardziej czasochłonna niż $\tau$.
5. Poniżej jest zaprezentowany zalążek kodu - jest on jedynie wskazówką i można na swój sposób zaimplementować tę metodę

In [9]:
from copy import deepcopy
from torch import nn
import torch.functional as F

class MLP(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, projector: bool = False):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(hidden_size),
            nn.Linear(hidden_size, output_size),
        )
        if projector:
            self.net.append(nn.BatchNorm1d(output_size))
            self.net.append(nn.ReLU(inplace=True))

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        return self.net(x)
    
class BYOL(nn.Module):
    def __init__(self, model, input_size=1000, hidden_size= 1024, projection_size=256, tau = 0.99) -> None:
        super().__init__()
        self.online_encoder = model
        self.online_projector = MLP(input_size, hidden_size, projection_size, projector= True)
        self.online_predictor = MLP(projection_size, hidden_size, projection_size, projector= False)
        self.online_net = nn.Sequential(
            self.online_encoder, 
            self.online_projector, 
        )

        self.target_encoder = self.deepcopy_and_freeze(self.online_encoder)
        self.target_projector = self.deepcopy_and_freeze(self.online_projector)
        self.target_net = nn.Sequential(self.target_encoder, self.target_projector)

        self.tau = tau

    def forward(self, x1, x2):
        z1 = self.online_net(x1)
        z2 = self.online_net(x2)
        p1 = self.online_predictor(z1)
        p2 = self.online_predictor(z2)
        return z1, z2, p1, p2

    def update_target(self):
        for online_params, target_params in zip(self.online_net.parameters(), self.target_net.parameters()):
            target_params.data = self.tau * target_params.data + (1 - self.tau) * online_params.data

    def loss(self, z1, z2, p1, p2):
        z2 = z2.detach()
        p2 = p2.detach()
        return 0.5 * (nn.functional.mse_loss(p1, z2) + nn.functional.mse_loss(p2, z1))
    
    @staticmethod
    def deepcopy_and_freeze(model: nn.Module) -> nn.Module:
        model_copy = deepcopy(model)
        for param in model_copy.parameters():
            param.requires_grad = False
        return model_copy    

In [10]:
def fit_byol(model, optimizer, train_dl, train_dl_prime, epochs=10, patience=5):
    metrics = {'train_loss': []}
    best_loss = float('inf')
    epochs_no_improve = 0
    early_stop = False

    for epoch in range(epochs):
        model.train()
        epoch_train_loss = 0
        total = 0

        for (x1, _), (x2, _) in tqdm(zip(train_dl, train_dl_prime), total=784):
            x1, x2 = x1.to(device), x2.to(device)
            optimizer.zero_grad()
            z1, z2, p1, p2 = model(x1, x2)
            loss = model.loss(z1, z2, p1, p2)
            loss.backward()
            optimizer.step()
            model.update_target()

            epoch_train_loss += loss.item() * x1.size(0)
            total += x1.size(0)

        train_loss = epoch_train_loss / total
        metrics['train_loss'].append(train_loss)
        print(f"Epoch {epoch}: train loss = {train_loss:.3f}")

        if train_loss < best_loss:
            best_loss = train_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                early_stop = True
                break

        if early_stop:
            print("Training stopped early")
            break

    return metrics

In [11]:
class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet18 = resnet18
        in_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(in_ftrs, num_classes)

    def forward(self, x):
        return self.resnet18(x)

In [12]:
t = torch.randn(512,3,96,96)
res = models.resnet18(pretrained=False)
t = res(t)

In [13]:
model = BYOL(models.resnet18(pretrained=False)).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

results = fit_byol(
    model,
    optimizer,
    transform_t_loader,
    transform_t_prime_loader,
    epochs=1,
    patience=7
)

100%|█████████▉| 782/784 [1:33:15<00:14,  7.16s/it]


Epoch 0: train loss = 0.014


## Douczanie nadzorowane

***Zaimplementuj*** proces douczania kodera z poprzedniego polecenia na danych etykietowanych ze zbioru treningowego. Porównaj jakość tego modelu z modelem nauczonym tylko na danych etykietownaych. Postaraj się wyjaśnić różnice.

In [14]:
state_dict = model.online_encoder.state_dict()
encoder = nn.Sequential(
    deepcopy(model.online_encoder),
    nn.Linear(1000,10)
).to(device)
# encoder.load_state_dict(state_dict)
supervised_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
fit(model=encoder, optimiser=supervised_optimizer, loss_fn=criterion, train_dl=train_dl, val_dl=test_dl, epochs=10, print_metrics=True)

100%|██████████| 40/40 [00:40<00:00,  1.02s/it]


Epoch 0: train loss = 0.022 (acc: 0.223), validation loss = 0.021 (acc: 0.225)


100%|██████████| 40/40 [00:39<00:00,  1.01it/s]


Epoch 1: train loss = 0.014 (acc: 0.300), validation loss = 0.014 (acc: 0.306)


100%|██████████| 40/40 [00:39<00:00,  1.01it/s]


...

Epoch 7: train loss = 0.011 (acc: 0.472), validation loss = 0.011 (acc: 0.445)


100%|██████████| 40/40 [00:39<00:00,  1.01it/s]


Epoch 8: train loss = 0.011 (acc: 0.490), validation loss = 0.012 (acc: 0.445)


100%|██████████| 40/40 [00:39<00:00,  1.01it/s]


Epoch 9: train loss = 0.010 (acc: 0.533), validation loss = 0.011 (acc: 0.481)
